In [96]:
import sys
# adding to the path variables the one folder higher (locally, not changing system variables)
sys.path.append("..")
import pandas as pd
import numpy as np
import warnings
import mlflow
from modeling.config import TRACKING_URI, EXPERIMENT_NAME
from pandas_profiling import ProfileReport
import torch

In [97]:
alert_df=pd.read_json("/Users/fortes/neuefische/Capstone-WIMT/data/alerts.json")

In [98]:
notification_df=pd.read_json("/Users/fortes/neuefische/Capstone-WIMT/data/notifications.json")

In [100]:
notification_df

,event_date,event_timestamp,event_name,user_id,join_field,join_key_value
0,20210529,1622251203899000,notification_receive,283a1,label,-Maph6wn8GA3SjPIc9HA
1,20210529,1622257988561000,notification_receive,283a1,label,-Maq63vqDUxQRawQ_3Au
2,20210529,1622258235184001,notification_receive,283a1,label,-Maq7-D7bhcEaFBU1LDW
3,20210529,1622260301025000,notification_receive,283a1,label,-MaqEsbKAqlxGG-RupjR
4,20210529,1622293962198000,notification_receive,283a1,label,-MasFHCt_lTuNt_x16dH
...,...,...,...,...,...,...
2278146,20210210,1612909178334000,notification_receive,fff18,label,0NbaiCRr6xGFd2035-Gdlw
2278147,20210210,1612969431222000,notification_receive,fff18,label,EMDjKLFr6xGRbrsa0gDjhg
2278148,20210210,1612978431552000,notification_receive,fff18,label,YE2VDMZr6xGRbrsa0gDjhg
2278149,20210210,1612985245425000,notification_receive,fff18,label,YBu-7NVr6xGRbrsa0gDjhg


In [101]:
notification_df.query("event_name == 'notification_opened'")

,event_date,event_timestamp,event_name,user_id,join_field,join_key_value
256,20210529,1622258577858000,notification_opened,e8300,source,notification_open
257,20210529,1622258577858000,notification_opened,e8300,alertId,null
597,20210529,1622254709611002,notification_opened,496f0,alertId,null
598,20210529,1622254709611002,notification_opened,496f0,source,notification_open
861,20210529,1622265938357003,notification_opened,8bb77,source,notification_open
...,...,...,...,...,...,...
2256035,20210706,1625581302874003,notification_opened,59258,source,notification_open
2256047,20210706,1625599828309000,notification_opened,59258,source,notification_open
2256048,20210706,1625599829755002,notification_opened,59258,source,notification_open
2256052,20210706,1625605040167003,notification_opened,59258,source,notification_open


In [102]:
#Look for what happened on these days.

In [109]:
#!pip3 install transformers --no-input

from transformers import pipeline

nlp_pos_syntax = pipeline(
    "ner",
    model="mrm8488/bert-spanish-cased-finetuned-pos-syntax",
    tokenizer="mrm8488/bert-spanish-cased-finetuned-pos-syntax"
)

text = 'Considera retrasos por servicios de emergencia atendiendo incendio en Mesones y Av. 20 de Noviembre.'

nlp_pos_syntax(text)[1:len(nlp_pos_syntax(text))-1]

In [125]:
cause_effect_table= pd.pivot_table(alert_df, index=['cause'], columns='effect',aggfunc='count',fill_value=0,values="is_global", margins=True, margins_name= 'Total')

In [126]:
cause_effect_table

effect,,CirculationRestored,CirculationShutdown,Delays,Evacuation,FullCapacity,HighWaitingTime,InterimService,LaneReduction,RouteDetour,SecuritySpeed,SuspensionOfService,TrafficImpact,Total
cause,,,,,,,,,,,,,,
BrokenTrafficLight,0,0,0,23,0,0,0,0,2,0,0,0,22,47
BrokenVehicle,0,0,4,390,0,0,0,0,39,0,0,12,55,500
COVID19,0,0,7,1,0,5,0,0,0,0,0,18,1,32
Counterflow,0,0,0,2,0,0,0,0,0,0,0,0,2,4
CycleRide,0,0,4,3,0,0,0,0,0,0,0,0,16,23
Demonstration,1,321,1896,166,0,0,0,65,54,5,6,251,226,2991
EmergencyServices,2,1,20,860,0,0,0,0,208,0,0,4,253,1348
Event,0,2,4,0,0,0,0,0,2,0,3,5,0,16
Explosion,0,0,0,0,0,0,0,0,0,0,0,0,1,1


In [127]:
notification_df["event_name"].unique()

array(['notification_receive', 'notification_dismiss',
       'notification_received', 'notification_open',
       'notification_opened', 'notification_share',
       'notification_view_alternatives'], dtype=object)

In [ ]:
#Rename elements on event name: receive and open]
[]

In [138]:
weird_table=notification_df.loc[(notification_df["join_field"]=="source")]
weird_table["join_key_value"].unique()

array(['notification', 'notification_open', 'line_status_detail_screen',
       'my_city_screen', 'locate_me_tab', 'notification_share',
       'notification_view_alternatives', 'travel_new_details_screen',
       'static_map_screen', 'journey_planner_itinerary_details_screen',
       'line_status_list_screen', 'settings_home_screen', 'my_city_tab',
       'alternative_journey_options_screen'], dtype=object)

In [139]:
weird_table

,event_date,event_timestamp,event_name,user_id,join_field,join_key_value
24,20210529,1622254038809000,notification_received,0f737,source,notification
29,20210529,1622260302438000,notification_received,0f737,source,notification
82,20210529,1622254036776000,notification_received,baafe,source,notification
254,20210529,1622258237428000,notification_received,e8300,source,notification
256,20210529,1622258577858000,notification_opened,e8300,source,notification_open
...,...,...,...,...,...,...
2256251,20210706,1625581262819000,notification_received,e0b4d,source,notification
2256257,20210706,1625584900674000,notification_received,e0b4d,source,notification
2256263,20210706,1625587521801000,notification_received,e0b4d,source,notification
2256278,20210706,1625528529057001,notification_received,427c1,source,notification


In [132]:
weird_table=notification_df.loc[(notification_df["join_field"]=="label")]
weird_table["join_key_value"].unique()

array(['-Maph6wn8GA3SjPIc9HA', '-Maq63vqDUxQRawQ_3Au',
       '-Maq7-D7bhcEaFBU1LDW', ..., 'VOZ8CMNr6xGRbrsa0gDjhg',
       '-QlfRa5r6xG1Dgex_M6_YA', 'WSvP595q6xGtKn2p0d1yrA'], dtype=object)

In [137]:
weird_table=notification_df.loc[(notification_df["join_field"]=="alertId")]
weird_table["join_key_value"].unique()

array(['-Maps-egC8fFLZM4HR5g', '-MaqEsbKAqlxGG-RupjR',
       '-Maq7-D7bhcEaFBU1LDW', ..., '-MdwYlzEEpRz31WrIpn5',
       '-MdwtPkCNIMmog9CCVcK', '-Mdxnjnltl0wZSXhawOz'], dtype=object)